# Guia Rápido - Cluster Hadoop

Este notebook fornece um guia interativo para configurar, testar e validar um cluster Apache Hadoop completo.

## Início Rápido (5 minutos)

### 1. Iniciar o Cluster

Execute o comando abaixo para iniciar todos os containers do cluster Hadoop.

In [ ]:
%%bash
docker-compose up -d

⏳ **Aguarde ~30 segundos** para inicialização completa dos serviços.

In [ ]:
import time
print("Aguardando inicialização do cluster...")
time.sleep(30)
print("✓ Cluster deve estar pronto!")

### 2. Verificar Status dos Containers

In [ ]:
%%bash
docker-compose ps

### 3. Acessar Interfaces Web

Abra os seguintes links no seu navegador:

- **HDFS NameNode**: [http://localhost:9870](http://localhost:9870)
- **YARN ResourceManager**: [http://localhost:8088](http://localhost:8088)
- **JobHistory Server**: [http://localhost:19888](http://localhost:19888)

### 4. Verificar Cluster Hadoop

#### Verificar DataNodes (HDFS)

In [ ]:
%%bash
docker exec hadoop-master hdfs dfsadmin -report

#### Verificar NodeManagers (YARN)

In [ ]:
%%bash
docker exec hadoop-master yarn node -list

✅ **Resultado Esperado:** Deve mostrar 2 DataNodes e 2 NodeManagers ativos.

### 5. Executar Exemplo WordCount

#### Copiar arquivos para o container

In [ ]:
%%bash
docker cp examples/wordcount hadoop-master:/tmp/

#### Executar job WordCount

In [ ]:
%%bash
docker exec hadoop-master bash -c "
  cd /tmp/wordcount
  chmod +x *.py *.sh
  ./run_wordcount.sh
"

### 6. Ver Resultados

#### Ver todos os resultados

In [ ]:
%%bash
docker exec hadoop-master hdfs dfs -cat /user/root/wordcount/output/part-*

#### Top 10 palavras mais frequentes

In [ ]:
%%bash
docker exec hadoop-master bash -c "
  hdfs dfs -cat /user/root/wordcount/output/part-* | sort -t$'\t' -k2 -nr | head -10
"

---

## 📚 Componentes do Projeto

Este projeto está dividido em três componentes principais, cada um abordando um aspecto diferente do Apache Hadoop:

### 1️⃣ Montagem de um Cluster Hadoop Básico (Configuração Básica)

**Objetivo:** Configurar e executar um cluster Hadoop funcional com Docker.

**Localização:**
- `docker-compose.yml` - Orquestração dos containers
- `hadoop-config/` - Arquivos de configuração do Hadoop
- `start-master.sh` e `start-worker.sh` - Scripts de inicialização

**O que foi implementado:**
- ✅ 1 nó master (NameNode + ResourceManager + JobHistory)
- ✅ 2 nós workers (DataNode + NodeManager)
- ✅ Interfaces web de monitoramento (portas 9870, 8088, 19888)
- ✅ HDFS com fator de replicação 2
- ✅ YARN configurado com 2GB por NodeManager
- ✅ MapReduce com JobHistory Server

**Documentação:** Ver `README.md` para detalhes completos da arquitetura.

#### Verificar configuração do cluster

In [ ]:
%%bash
echo "=== Status do Cluster ==="
docker-compose ps

echo -e "\n=== DataNodes (HDFS) ==="
docker exec hadoop-master hdfs dfsadmin -report | grep -A 2 "Live datanodes"

echo -e "\n=== NodeManagers (YARN) ==="
docker exec hadoop-master yarn node -list

---

### 2️⃣ Teste de Comportamento do Framework Hadoop

**Objetivo:** Demonstrar como diferentes configurações impactam performance e comportamento do HDFS, YARN e MapReduce.

**Localização:** `tests/`

**5 Testes Implementados:**

1. **test1_replication.sh** - Fator de replicação HDFS (1, 2, 3)
   - Impacto no uso de disco
   - Distribuição de blocos entre DataNodes
   - Trade-off entre segurança e espaço

2. **test2_yarn_memory.sh** - Memória YARN (1GB, 2GB, 4GB)
   - Número de containers simultâneos
   - Performance de jobs
   - Utilização de recursos

3. **test3_scheduler_queues.sh** - Filas do Capacity Scheduler
   - Single queue vs multiple queues (high/default/low)
   - Priorização de jobs
   - Isolamento de recursos

4. **test4_block_size.sh** - Tamanho de blocos HDFS (64MB, 128MB, 256MB)
   - Número de map tasks geradas
   - Overhead de metadados no NameNode
   - Performance de I/O

5. **test5_mapreduce_memory.sh** - Memória de containers MapReduce
   - Memória para mappers e reducers (256MB, 512MB, 1024MB)
   - Paralelismo vs consumo de recursos
   - Otimização de performance

**Resultados:** Arquivos salvos em `tests/results/` com métricas detalhadas e análise comparativa.

**Documentação:** Ver `tests/README.md` e `tests/TESTING_GUIDE.md` para detalhes de cada teste.

#### Executar teste individual (Teste 1: Replicação HDFS)

In [ ]:
%%bash
./tests/scripts/test1_replication.sh baseline

#### Executar todos os testes (~35-40 minutos)

⚠️ **Aviso:** Isso levará bastante tempo. Execute apenas se tiver tempo disponível.

In [ ]:
%%bash
# Descomente a linha abaixo para executar todos os testes
# ./tests/run_all_tests.sh

#### Gerar relatório consolidado

In [ ]:
%%bash
./tests/generate_report.sh

---

### 3️⃣ Teste de Tolerância a Falhas e Performance

**Objetivo:** Avaliar resiliência do Hadoop sob condições adversas e medir capacidade de recuperação.

**Localização:** `fault-tolerance/`

**4 Cenários de Teste:**

1. **Baseline** - Performance sem falhas
   - Cluster completo (2 workers)
   - Execução normal de WordCount
   - Estabelece linha de base de tempo (~3-4 min)

2. **Worker Failure** - Falha de 1 worker durante execução
   - Remove hadoop-worker1 após 30s
   - Testa recuperação automática do YARN
   - Job deve completar com ~20-40% mais tempo

3. **Scale Up** - Adição dinâmica de worker
   - Inicia com 1 worker apenas
   - Adiciona worker2 após 30s
   - Demonstra elasticidade do cluster

4. **Multiple Failures** - Falhas múltiplas (catastrófico)
   - Remove ambos workers progressivamente
   - Job deve FALHAR
   - Identifica limites de tolerância

**Scripts disponíveis:**
- `generate_data.sh` - Gera dataset de 500MB+ para jobs longos
- `upload_data.sh` - Upload para HDFS com verificação
- `monitor_job.sh` - Monitora jobs em tempo real
- `run_fault_test.sh` - Orquestra os 4 testes
- `generate_report.sh` - Relatório consolidado com análise

**Métricas coletadas:**
- Tempo de execução e recuperação
- Taxa de sucesso/falha
- Comportamento do ResourceManager
- Logs detalhados de cada cenário

**Documentação:** Ver `fault-tolerance/README.md` para análise completa dos resultados esperados.

#### Passo 1: Gerar dados de teste (500MB)

In [ ]:
%%bash
./fault-tolerance/scripts/generate_data.sh

#### Passo 2: Upload para HDFS

In [ ]:
%%bash
./fault-tolerance/scripts/upload_data.sh

#### Passo 3a: Executar teste baseline

In [ ]:
%%bash
./fault-tolerance/scripts/run_fault_test.sh baseline

#### Passo 3b: Executar teste de falha de worker

In [ ]:
%%bash
./fault-tolerance/scripts/run_fault_test.sh worker-failure

#### Passo 3c: Executar teste de scale up

In [ ]:
%%bash
./fault-tolerance/scripts/run_fault_test.sh scale-up

#### Passo 3d: Executar teste de falhas múltiplas

In [ ]:
%%bash
./fault-tolerance/scripts/run_fault_test.sh multiple-failures

#### Ou executar todos os testes de tolerância a falhas

⚠️ **Aviso:** Isso levará aproximadamente 15-20 minutos.

In [ ]:
%%bash
# Descomente a linha abaixo para executar todos os testes
# ./fault-tolerance/scripts/run_fault_test.sh all

#### Passo 4: Gerar relatório de tolerância a falhas

In [ ]:
%%bash
./fault-tolerance/scripts/generate_report.sh

---

## 🎯 Fluxo Recomendado de Execução

Para executar o projeto completo na ordem correta:

**Tempo total estimado:** ~50-60 minutos para todos os testes

### Passo 1: Montar cluster básico (✅ Já executado acima)

In [ ]:
%%bash
docker-compose up -d
sleep 30
docker exec hadoop-master hdfs dfsadmin -report | grep -A 2 "Live datanodes"

### Passo 2: Executar testes de comportamento (~35-40 min)

In [ ]:
%%bash
# Descomente para executar
# ./tests/run_all_tests.sh
# ./tests/generate_report.sh

### Passo 3: Testes de tolerância a falhas (~15-20 min)

In [ ]:
%%bash
# Descomente para executar
# ./fault-tolerance/scripts/generate_data.sh
# ./fault-tolerance/scripts/upload_data.sh
# ./fault-tolerance/scripts/run_fault_test.sh all
# ./fault-tolerance/scripts/generate_report.sh

---

## Comandos Úteis

### Gerenciar Cluster

In [ ]:
%%bash
# Iniciar
docker-compose up -d

In [ ]:
%%bash
# Parar
docker-compose down

In [ ]:
%%bash
# Ver logs
docker logs hadoop-master --tail 50

### Trabalhar com HDFS

In [ ]:
%%bash
# Criar diretório
docker exec hadoop-master hdfs dfs -mkdir -p /user/root/meudir

In [ ]:
%%bash
# Listar arquivos
docker exec hadoop-master hdfs dfs -ls /user/root/

In [ ]:
%%bash
# Upload arquivo (exemplo)
docker exec hadoop-master bash -c "echo 'teste' > /tmp/teste.txt"
docker exec hadoop-master hdfs dfs -put /tmp/teste.txt /user/root/meudir/

In [ ]:
%%bash
# Ver conteúdo
docker exec hadoop-master hdfs dfs -cat /user/root/meudir/teste.txt

### Monitorar Jobs YARN

In [ ]:
%%bash
# Listar aplicações em execução
docker exec hadoop-master yarn application -list

In [ ]:
%%bash
# Status de uma aplicação (substitua APPLICATION_ID)
# docker exec hadoop-master yarn application -status <APPLICATION_ID>

---

## Estrutura do Projeto

```
trabalho_hadoop/
├── docker-compose.yml              # Configuração do cluster
├── hadoop-config/                  # Arquivos de configuração
│   ├── core-site.xml              # Configurações gerais
│   ├── hdfs-site.xml              # HDFS
│   ├── yarn-site.xml              # YARN
│   ├── mapred-site.xml            # MapReduce
│   ├── capacity-scheduler.xml     # Scheduler
│   └── workers                    # Lista de workers
├── examples/
│   └── wordcount/                 # Exemplo WordCount
├── tests/                         # Testes de comportamento
│   ├── scripts/                   # 5 scripts de teste
│   ├── results/                   # Resultados
│   ├── README.md
│   └── TESTING_GUIDE.md
├── fault-tolerance/               # Testes de tolerância a falhas
│   ├── scripts/                   # Scripts de teste
│   ├── data/                      # Dados gerados
│   ├── results/                   # Resultados
│   └── README.md
├── README.md                       # Documentação completa
├── QUICK_START.md                 # Guia rápido (texto)
└── Hadoop_Quick_Start.ipynb       # Este notebook
```

## Recursos do Cluster

- **Nós**: 1 master + 2 workers
- **Memória YARN**: 4 GB total (2 GB/worker)
- **CPUs**: 4 vCPUs (2/worker)
- **Replicação HDFS**: 2 réplicas
- **Hadoop Version**: 3.3.6

## Solução Rápida de Problemas

### DataNodes não conectam

In [ ]:
%%bash
# Reiniciar cluster
docker-compose restart
sleep 30

# Ver logs
docker logs hadoop-master --tail 20

### Limpar tudo e recomeçar

In [ ]:
%%bash
# Parar e remover volumes
docker-compose down -v

# Reiniciar
docker-compose up -d

## Próximos Passos

1. ✅ Cluster funcionando
2. ✅ Exemplo WordCount executado
3. 📝 Executar testes de comportamento
4. 📝 Executar testes de tolerância a falhas
5. 📝 Analisar relatórios gerados
6. 📝 Tirar screenshots das interfaces web
7. 📝 Documentar resultados para entrega

## 📦 Entrega do Trabalho

Para documentar sua entrega, inclua:

1. ✅ Arquivos de configuração (`hadoop-config/`)
2. ✅ Docker Compose configurado
3. ✅ Screenshots das interfaces web
4. ✅ Exemplo de job executado
5. ✅ Resultados dos testes
6. ✅ Relatórios gerados

Tire screenshots de:
- http://localhost:9870 (HDFS com 2 DataNodes)
- http://localhost:8088 (YARN com jobs executados)
- http://localhost:19888 (JobHistory)
- Resultados dos testes de comportamento
- Resultados dos testes de tolerância a falhas

---

## 🎉 Conclusão

Você agora tem:
- ✅ Um cluster Hadoop completo e funcional
- ✅ Framework de testes de comportamento (5 testes)
- ✅ Framework de testes de tolerância a falhas (4 cenários)
- ✅ Documentação completa e interativa

Execute as células acima na ordem para configurar e testar seu cluster!

**Boa sorte! 🚀**